# Local tests (NO POSTGRES)

In [22]:
%run src/cdc_pdf_to_text.py CDC.pdf CDC.txt

Structured output written to CDC.txt


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load SBERT model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

paragraphs = []

with open("CDC.txt", "r", encoding="utf-8") as f:
    chapter = None
    paragraphs = f.read().split("\n")

# Embed
embeddings = model.encode(paragraphs)

In [21]:
query = "Produto com defeito"
# Embed query
query_embedding = model.encode(query)
results = util.semantic_search(query_embedding, embeddings, score_function=util.dot_score)

for result in results[0]:
    print(paragraphs[result['corpus_id']])

CAPITULO IV - Art. 12. O fabricante, o produtor, o construtor, na-  cional ou estrangeiro, e o importador respondem, inde- pendentemente da existência de culpa, pela reparação  dos danos causados aos consumidores por defeitos  decorrentes de projeto, fabricação, construção, monta- gem, fórmulas, manipulação, apresentação ou acondi- 12 cionamento de seus produtos, bem como por informa- ções insuficientes ou inadequadas sobre sua utilização   e riscos. 	 § 1º O produto é defeituoso quando não oferece a  segurança que dele legitimamente se espera, levando- -se em consideração as circunstâncias relevantes, entre  as quais: 	 I - sua apresentação; 	 II - o uso e os riscos que razoavelmente dele se  esperam; 	 III - a época em que foi colocado em circulação. 	 § 2º O produto não é considerado defeituoso pelo  fato de outro de melhor qualidade ter sido colocado no  mercado. 	 § 3º O fabricante, o construtor, o produtor ou impor- tador só não será responsabilizado quando provar: 	 I - que não 

# Docker tests (with Postgres)

In [39]:
!./start.sh

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[+] Building 0.0s (0/1)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
[+] Building 0.2s (14/14) FINISHED                         docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 652B                                       0.0s
 => WARN: SecretsUsedInArgOrEnv: Do not use ARG or ENV instructions for s  0.0s
 => WARN: SecretsUsedInArgOrEnv: Do not use ARG or ENV instructions for s  0.0s
 => [internal] load metadata for docker.io/pgvector/pgvector:pg17          0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [1/9] FROM docker.io/pgvector/pgvector:pg17@sha256:5982c00a2cdf786c2d  0.0s
 => => resolve docker.io/pgvector/pgvector:pg17@sha256:5982c00a2cdf786c2d  0.0s
 => [internal] load build context       

In [40]:
import psycopg2

import os

query = "Produto com defeito"
# Embed query
query_embedding = model.encode(query)

os.environ['POSTGRES_USER'] = 'admin'
os.environ['POSTGRES_PASSWORD'] = 'admin'
os.environ['POSTGRES_DB'] = 'cdc'

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname=os.environ['POSTGRES_DB'],
    user=os.environ['POSTGRES_USER'],
    password=os.environ['POSTGRES_PASSWORD'],
    host="127.0.0.1",
    port="5432"
)

# Create a cursor object
cur = conn.cursor()

# Perform a search using pgvector
query_vector = query_embedding.tolist()  # Convert the numpy array to a list
sql = """
SELECT id, content, embedding <=> %s::vector AS distance
FROM documents
ORDER BY distance ASC
LIMIT 10;
"""
cur.execute(sql, (query_vector,))

# Fetch and print the results
rows = cur.fetchall()
for row in rows:
    print(f"{row[1]}")

# Close the cursor and connection
cur.close()
conn.close()

CAPITULO IV - Art. 12. O fabricante, o produtor, o construtor, na-  cional ou estrangeiro, e o importador respondem, inde- pendentemente da existência de culpa, pela reparação  dos danos causados aos consumidores por defeitos  decorrentes de projeto, fabricação, construção, monta- gem, fórmulas, manipulação, apresentação ou acondi- 12 cionamento de seus produtos, bem como por informa- ções insuficientes ou inadequadas sobre sua utilização   e riscos. 	 § 1º O produto é defeituoso quando não oferece a  segurança que dele legitimamente se espera, levando- -se em consideração as circunstâncias relevantes, entre  as quais: 	 I - sua apresentação; 	 II - o uso e os riscos que razoavelmente dele se  esperam; 	 III - a época em que foi colocado em circulação. 	 § 2º O produto não é considerado defeituoso pelo  fato de outro de melhor qualidade ter sido colocado no  mercado. 	 § 3º O fabricante, o construtor, o produtor ou impor- tador só não será responsabilizado quando provar: 	 I - que não 